In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook

depeches = pd.read_csv("depeches.csv")
photos = pd.read_csv("photos.csv",encoding='latin-1')

c:\users\maison\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
c:\users\maison\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\Maison\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
print(depeches.columns,'\n',photos.columns)

Index(['newsItemID', 'event', 'news2'], dtype='object') 
 Index(['Unnamed: 0', 'Uno', 'caption', 'city', 'contributor', 'country',
       'country_only', 'country_out', 'created', 'creator', 'dept',
       'entity_company', 'entity_departement', 'entity_faces',
       'entity_function', 'entity_keyword', 'entity_location', 'entity_media',
       'entity_organisation', 'entity_person', 'entity_region', 'entity_video',
       'event', 'expires', 'genre', 'headline', 'iptc', 'keyword', 'language',
       'mediatopic', 'newsItemID', 'product', 'provider', 'published_date',
       'region', 'revision', 'rules', 'signal', 'slug', 'source', 'status',
       'title', 'topic', 'urgency', 'url_extracted'],
      dtype='object')


In [3]:
print(depeches.shape)
depeches.event

(19975, 3)


0                                       ['afpevent:FHL21']
1                                       ['afpevent:FNB22']
2                                       ['afpevent:FOA15']
3                     ['afpevent:EDU58', 'afpevent:FFQ36']
4                                       ['afpevent:FHL21']
5                                       ['afpevent:FOA79']
6                                       ['afpevent:FOA86']
7        ['afpevent:FNR78', 'afpevent:FCS97', 'afpevent...
8                                       ['afpevent:FHE29']
9                                       ['afpevent:FOA12']
10                                      ['afpevent:FOA55']
11                                      ['afpevent:FAR96']
12                                      ['afpevent:FAR96']
13                                      ['afpevent:FAR96']
14       ['afpevent:FFQ37', 'afpevent:EDU79', 'afpevent...
15                                      ['afpevent:FHR22']
16                                      ['afpevent:DZZ87

In [4]:
print('Il y a ', np.sum(depeches.event.isna()), " dépêches dont l'événement n'est pas renseigné.", sep ='')

Il y a 0 dépêches dont l'événement n'est pas renseigné.


In [5]:
print(photos.shape)
photos.event

(360999, 45)


0                           ['afpevent:FKF04']
1                                          NaN
2                                          NaN
3                                          NaN
4                                          NaN
5                                          NaN
6                                          NaN
7                                          NaN
8                                          NaN
9                                          NaN
10                                         NaN
11                                         NaN
12                          ['afpevent:FKF04']
13                          ['afpevent:FKF04']
14        ['afpevent:FNG29', 'afpevent:FNG84']
15                                         NaN
16                                         NaN
17                                         NaN
18                                         NaN
19                                         NaN
20                          ['afpevent:FKF04']
21           

In [6]:
print('Il y a ', np.sum(photos.event.isna()), " articles dont l'événement n'est pas renseigné.", sep ='')

Il y a 115524 articles dont l'événement n'est pas renseigné.


In [7]:
liste_ev_unique_depeches = []
for ev in depeches.event:
    for _ev in ev[1:-1].replace("'","").split(', '):
        if _ev not in liste_ev_unique_depeches:
            liste_ev_unique_depeches.append(_ev)
print(len(liste_ev_unique_depeches))

13335


In [8]:
liste_ev_unique_depeches

['afpevent:FHL21',
 'afpevent:FNB22',
 'afpevent:FOA15',
 'afpevent:EDU58',
 'afpevent:FFQ36',
 'afpevent:FOA79',
 'afpevent:FOA86',
 'afpevent:FNR78',
 'afpevent:FCS97',
 'afpevent:FCQ20',
 'afpevent:FHE29',
 'afpevent:FOA12',
 'afpevent:FOA55',
 'afpevent:FAR96',
 'afpevent:FFQ37',
 'afpevent:EDU79',
 'afpevent:EDZ99',
 'afpevent:FHR22',
 'afpevent:DZZ87',
 'afpevent:FGT28',
 'afpevent:FHT00',
 'afpevent:FHN08',
 'afpevent:FOA32',
 'afpevent:FNT17',
 'afpevent:FNL61',
 'afpevent:FOA22',
 'afpevent:FOA36',
 'afpevent:FOA31',
 'afpevent:FLB15',
 'afpevent:FNZ94',
 'afpevent:FNZ78',
 'afpevent:FOA10',
 'afpevent:FOA01',
 'afpevent:FJI21',
 'afpevent:FNZ80',
 'afpevent:FNN89',
 'afpevent:FIC06',
 'afpevent:FNZ84',
 'afpevent:FOA06',
 'afpevent:FNZ87',
 'afpevent:FNQ42',
 'afpevent:FNZ97',
 'afpevent:FNZ90',
 'afpevent:FNZ89',
 'afpevent:FNZ95',
 'afpevent:FNZ91',
 'afpevent:FNZ88',
 'afpevent:FIL37',
 'afpevent:FLJ68',
 'afpevent:FJJ24',
 'afpevent:FJJ26',
 'afpevent:FJJ27',
 'afpevent:F

In [9]:
liste_ev_unique_photos = []
for ev in photos.event:
    if(type(ev) != float):
        for _ev in ev[1:-1].replace("'","").split(', '):
            if _ev not in liste_ev_unique_photos:
                liste_ev_unique_photos.append(_ev)
print(len(liste_ev_unique_photos))

16059


In [10]:
liste_ev_unique_photos

['afpevent:FKF04',
 'afpevent:FNG29',
 'afpevent:FNG84',
 'afpevent:FMZ53',
 'afpevent:DYT14',
 'afpevent:FGE57',
 'afpevent:FBI68',
 'afpevent:FIY20',
 'afpevent:FDY34',
 'afpevent:FDT08',
 'afpevent:FDY29',
 'afpevent:FNJ85',
 'afpevent:EUU72',
 'afpevent:FDF07',
 'afpevent:FMJ94',
 'afpevent:FLZ86',
 'afpevent:FNJ37',
 'afpevent:FMW94',
 'afpevent:FMH31',
 'afpevent:DRR98',
 'afpevent:FNG10',
 'afpevent:FNF28',
 'afpevent:FKU10',
 'afpevent:JV91',
 'afpevent:FHL21',
 'afpevent:FNJ95',
 'afpevent:FDT76',
 'afpevent:FDT74',
 'afpevent:FDS74',
 'afpevent:FNK38',
 'afpevent:FIQ44',
 'afpevent:FNI96',
 'afpevent:FMZ39',
 'afpevent:FNG42',
 'afpevent:FNK14',
 'afpevent:FLP85',
 'afpevent:FLR27',
 'afpevent:FNJ65',
 'afpevent:FNF09',
 'afpevent:FNF12',
 'afpevent:FNF39',
 'afpevent:FNC50',
 'afpevent:FCL46',
 'afpevent:FNK16',
 'afpevent:ELI39',
 'afpevent:FNE62',
 'afpevent:EXH24',
 'afpevent:FGZ77',
 'afpevent:FDY31',
 'afpevent:FMW09',
 'afpevent:FLU06',
 'afpevent:FFG54',
 'afpevent:FN

In [11]:
ev_out = 0
for ev in liste_ev_unique_depeches:
    if(ev not in liste_ev_unique_photos):
        ev_out += 1
print(ev_out, 'événements ne correspondent pas.')

10355 événements ne correspondent pas.


In [12]:
ind = []
for i in range(depeches.shape[0]):
    score = 0
    liste_ev = depeches.event[i][1:-1].replace("'","").split(', ')
    for ev in liste_ev:
        if(ev in liste_ev_unique_photos):
            score += 1
    if(score > 0):
        ind.append(i)

In [13]:
len(ind)

10197

In [14]:
ind_photos = []
for i in range(photos.shape[0]):
    if(type(photos.event[i]) != float):
        score = 0
        liste_ev = photos.event[i][1:-1].replace("'","").split(', ')
        for ev in liste_ev:
            if(ev in liste_ev_unique_depeches):
                score += 1
        if(score > 0):
            ind_photos.append(i)

In [15]:
len(ind_photos)

77436

In [16]:
rand_ind_list=[]
rand_eve_list=[]
for i in range(100):
    num = random.randint(0, len(ind)-1)
    while(ind[num] in rand_ind_list or set(depeches.event[ind[num]][1:-1].replace("'","").split(', ')).intersection(rand_eve_list) != set()):
        num = random.randint(0, len(ind)-1)
    rand_ind_list.append(ind[num])
    for ev in depeches.event[ind[num]][1:-1].replace("'","").split(', '):
        rand_eve_list.append(ev)
print(rand_ind_list)

[1665, 6320, 17996, 9429, 6290, 9368, 11613, 14360, 617, 14005, 16162, 14607, 8556, 17690, 10065, 6141, 2388, 19208, 13117, 17005, 17743, 4080, 7509, 16917, 5702, 1848, 8072, 7777, 11875, 13713, 9237, 13729, 18810, 5282, 8372, 3553, 3481, 14580, 10266, 12504, 13640, 10576, 10387, 14804, 14821, 18690, 6935, 3582, 3082, 9427, 19338, 9330, 9186, 6815, 9049, 6824, 2379, 2049, 6258, 15749, 11822, 2077, 16431, 12696, 578, 11615, 17429, 15317, 17293, 2079, 16499, 7314, 13081, 4637, 17719, 16705, 18044, 11796, 14108, 12456, 5070, 7340, 16403, 1144, 16998, 9119, 16235, 11285, 6054, 1441, 12233, 16094, 16515, 13625, 8530, 1882, 19398, 8102, 11295, 17596]


In [117]:
rand_ind_list_photos=[]
for j in range(100):
    ensemble = set(depeches.event[rand_ind_list[j]][1:-1].replace("'","").split(', '))
    T_true=[ind_photos[i] for i in range(len(ind_photos)) if set(photos.event[ind_photos[i]][1:-1].replace("'","").split(', ')).intersection(ensemble)
              != set()]
    if(len(T_true) < 10):
        rand_ind_list_photos += T_true
    else:
        for i in range(15):
            num = random.randint(0, len(T_true)-1)
            rand_ind_list_photos.append(T_true[num])
rand_ind_list_photos = list(set(rand_ind_list_photos))
print(len(rand_ind_list_photos))

1012


In [62]:
#Toute cette cellule est issue de l'autre notebook (création du csv) et permet donc d'avoir toutes les fonctions
#définies dans ce dernier à disposition.


#On importe le fichier contenant les scores de chaque fonction grammaticale
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

#On importe un modèle md pour avoir des mots vectorisés
nlp=spacy.load("en_core_web_md")  
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

def dep_ent(ent, doc):
    """ Retourne la fonction grammaticale :  la 'dep', d'une entité. Cette fonction est nécessaire car elle permet d'affecter
    une dep à une entité composée de plusieurs mots ayant chacun une dep de base.
    Traite aussi le cas particulier des mots étant des conj ou des compound : leur vrai dep et celle du mot auxquels
    ils sont associés en tant que conj ou compound."""
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':     
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      
                return(tok.dep_)
            return(doc[k].dep_)    
    return doc[start].dep_

def ent_good_type(ent): #filtre les entités selon leur type
    return (ent.label_ == "PERSON"or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC")

def sort_ent(doc):
    """Retourne la liste des entités en les filtrant selon leur type et en les triant de manière à avoir au début de
    la liste les entités ayant des coréférences."""
    ent_coref=[ent for ent in doc.ents if ent._.is_coref and ent_good_type(ent)]
    ent_vanilla=[ent for ent in doc.ents if ent_good_type(ent) and not ent._.is_coref]
    return ent_coref + ent_vanilla

def scores_doc(doc):
    """Retourne le score de chaque entité pour la méthode sans neuralcoref."""
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def scores_doc_coref1(doc):
    """Retourne le score de chaque entité pour la méthode de base opérée sur le document resolved.
    Le document resolved est le document de base dans lequel toutes les références à un groupe de mot sont remplacées
    par celui-ci.
    Par exemple : My dad is home. He watches TV devient My dad is home. My dad watches TV.
    En raisonnant avec le nlp sur le document resolved, le nlp va détecter beaucoup plus de fois la même entité.
    Le principal inconvénient est que toutes les références sont remplacées, y compris celles qui ne sont pas associées 
    à des entités mais plutôt à des très longs bouts de phrase qui sont repris par un pronom comme "it"  """
    
    doc=nlp(doc._.coref_resolved)
    res={}
    for ent in doc.ents:
        if ent_good_type(ent):
            if ent.text not in res.keys():
                res[ent.text]=dict_val[dep_ent(ent,doc)]
            else:
                res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def is_in_cluster(ent,cluster):  #détermine si une entité est dans une des mentions d'un cluster
    for span in cluster.mentions:
        if ent.text in span.text:
            return True
    return False



def scores_doc_coref2(doc):
    """Cette méthode utilise neuralcoref mais au lieu d'agir sur le doc resolved, on va chercher a l'interieur des clusters
    associés au document. Pour rappel, un cluster contient un mot et ses références. Ce mot n'est pas forcément une entité.
    Un cluster est de la forme Trump : [Trump, he].
    Pour chaque entité, on va d'abord regarder si elle est coréférencée (donc mentionnée explicitement dans un cluster)
    ou pas.
    Si elle est coréférencée, on va chercher le cluster qui lui est associé.
    Une fois dans ce cluster, on va regarder pour chaque mention (réf à l'entité) si elle contient ou pas des entités.
    Si c'est le cas, on va alors indiquer que cette entité ne doit pas être retraitée par la suite car elle est traitée en
    tant qu'autre mention de l'entité que l'on traite actuellement.
    Par exemple, pour l'entité référencée Trump, on trouve le cluster suivant : Trump : [Trump, Donald Trump, he].
    Alors l'entité Donald Trump va être marquée comme traitée car elle est en fait équivalente à Trump.
    "he" n'étant pas une entité on a pas ce pb pour cette mention la.
    Ensuite on calcule le score de chaque mention (Trump, Donald Trump et he) en se basant sur leur fonction grammaticale.
    Puis on ajoute ces scores à celui de l'entité de base (Trump).
    
    Si l'entité de base n'est pas référencée, c'est un peu le même principe, la seule complexité en plus étant le fait que
    l'entité n'est pas forcément mentionnée telle quelle dans les mentions d'un cluster. 
    Par exemple, si notre entité de base est Sienna Miller, il est possible qu'on ait un cluster de la forme
    The british actress Sienna Miller : [The british actress Sienna Miller, she].
    Dans ce cas il faut rechercher si chaque mention contient l'entité de base avant de procéder comme pour les 
    entités référencées.
    
    """
    clusters=doc._.coref_clusters  #liste des clusters du doc. 
    res={} #le resultat sera un dictionnaire qui permet d'associer un score à chque entité
    ent_treated={} #dictionnaire pour différencier les entités traitées des autres
    ents_sorted=sort_ent(doc)  #on place les entités ayant une coref avant les autres
    for ent in ents_sorted:   
        ent_treated[ent.text]=0  #on initialise en affectant 0 à toutes les entités

    for ent in ents_sorted:     #pour chaque entité
        if ent_treated[ent.text]==0: #si elle n'est pas considérée comme "traitée"
            if ent._.is_coref:  #si elle est coréférencée (donc si elle apparait en tant que mention pour un cluster )
                #print(ent)
                for cluster in clusters:  #on va chercher le cluster associé à l'entité puisqu'elle est coref
                    if ent in cluster.mentions: # on regarde si l'entité est dans les mentions du cluster
                        for span in cluster.mentions: #Mtnt qu'on est dans le bon cluster, on regarde pour chaque span
                            if span.ents != [] : 
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents: #on cherche l'entité dans le span la plus proche de ent
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1 #une fois qu'on l'a trouvé on la marque comme traitée
                            else:
                                max_span_ent=span        
                            if ent.text not in res.keys(): 
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)]  #on affecte le score de l'entité
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)] 
            else: #si l'entité n'est pas exactement coréférencée par neuralcoref
                flag=0
                for cluster in clusters:
                    if is_in_cluster(ent,cluster) and ent.label_ != 'NORP': 
                        flag=1
                        for span in cluster.mentions:
                            #print(span)
                            if span.ents != [] :
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents:
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1
                            if ent.text not in res.keys():
                                res[ent.text]=dict_val[dep_ent(span,doc)] #...replacer max_span_ent par span ici
                            else:
                                res[ent.text] += dict_val[dep_ent(span,doc)]  #et ici
                if flag==0: #si l'entité n'est vraiment dans aucun cluster   
                    if ent.text not in res.keys():
                        res[ent.text]=dict_val[dep_ent(ent,doc)]
                    else:
                        res[ent.text]+=dict_val[dep_ent(ent,doc)] 
    return res


def freq_dict(L):
    n = len(L)
    L_unique = list(set(L))
    d = {}
    for a in L_unique:
        for b in L:
            if a == b:
                if a in d.keys():
                    d[a] += 1/n
                else:
                    d[a] = 1/n
    return(d)

In [222]:
def tri_dict(d): 
    """Tri décroissant """

    d_sort = {}
    for k,v in sorted(d.items(), key =lambda x :x[1], reverse= True):
        d_sort[k]=v
    return d_sort

def score_sim11(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Rajout d'une pénalité par entité différentes 
    """
    res=0
    freq_doc=freq_dict([ent.text for ent in doc.ents])
    score_image=liste_scores1[k]
    freq_image=liste_freq[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 #- abs(freq_doc[i]-freq_image[i])/2
    score_image_tri = tri_dict(score_image)
    c = 0 
    for i in score_image_tri.keys():
        c+=1
        if i not in score_doc.keys():
            
            res -= 1/(c+1)**2
        
    return res

In [223]:
def score_sim3(score_doc,score_image):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Par rapport aux fonctions précédentes, ici on a changé le calcul du score de similarité en "normalisant" les scores
    du doc et de l'image, au lieu de les diviser par 2 (ce qui était inutile)"""
    res=0
    max_doc=max(score_doc.values())
    max_im=max(score_image.values())
    for i in score_doc.keys():
        if i in score_image.keys():
            if max_doc==0 or max_im==0:
                return 0
            res += score_doc[i]/max_doc +score_image[i]/max_im
    score_image_tri = tri_dict(score_image)
    c = 0 
    for i in score_image_tri.keys():
        c+=1
        if i not in score_doc.keys():
            
            res -= 1/(c+1)**2
    return res

In [119]:
nlp_depeches = [nlp(depeches.news2[i]) for i in rand_ind_list]
nlp_photos = [nlp(photos.caption[i]) for i in rand_ind_list_photos]

In [224]:
scores_depeches = [scores_doc_coref1(dep) for dep in nlp_depeches]
scores_photos = [scores_doc_coref1(pho) for pho in nlp_photos]

In [225]:
n = len(scores_depeches)
nn = 0
score = 0
prob = []
for i in range(n):
    if(len(scores_depeches[i].values()) != 0):
        nn += 1
        sco = [(score_sim3(scores_depeches[i], scores_photos[j]),j) for j in range(len(scores_photos)) if len(scores_photos[j].values()) != 0]
        indice_argmax = np.argmax(sco, axis = 0)[0]
        if(set(photos.event[rand_ind_list_photos[sco[indice_argmax][1]]][1:-1].replace("'","").split(', ')).intersection(set(depeches.event[rand_ind_list[i]][1:-1].replace("'","").split(', '))) != set()):
            score += 1
        else:
            prob.append(i)
print(score/nn)

0.7


In [165]:
prob[0]
scores_depeches[2]

{'Ahmed bin Saeed Al-Maktoum': 1.0,
 'the Dubai Airshow': 0.872340426,
 'Emirates': 9.468085107,
 'A350': 1.0,
 'Dubai': 0.872340426,
 'the Middle East': 0.872340426}

In [169]:
depeches.news2[rand_ind_list[2]]

'Emirates Airline said Monday it would buy 50 Airbus 350-900 widebody aircraft in a $16 billion deal, as it reorganises its fleet after cutting orders of the A380 superjumbo.\n "I am happy to announce we are purchasing 50 Airbus 350 for $16 billion," the carrier\'s chairman and CEO Sheikh Ahmed bin Saeed Al-Maktoum told reporters at the Dubai Airshow, adding that delivery would begin from May 2023.\n Its move earlier this year to cut 39 aircraft from its total A380 orders prompted Airbus to pull the plug on the costly plane, which airlines have struggled to fill to its capacity of 500-850 people.\n The airline said that Monday\'s announcement "replaces the heads of agreement signed in February where Emirates announced its intent to purchase 30 A350s and 40 A330Neos."\n The airline said the A350 will enable it to serve a range of new markets, including on long-haul routes up to 15 hours of flying time from its base in Dubai.\n Emirates Airline, the largest carrier in the Middle East, re

In [201]:
test = [(score_sim3(scores_depeches[2], scores_photos[j]),j) for j in range(len(scores_photos)) if len(scores_photos[j].values()) != 0]

In [202]:
np.argmax(test, axis = 0)[0]

957

In [206]:
scores_depeches[2]

{'Ahmed bin Saeed Al-Maktoum': 1.0,
 'the Dubai Airshow': 0.872340426,
 'Emirates': 9.468085107,
 'A350': 1.0,
 'Dubai': 0.872340426,
 'the Middle East': 0.872340426}

In [208]:
text = depeches.news2[rand_ind_list[2]]

In [212]:


text.find('Emirates')

0

In [218]:
text

'Emirates Airline said Monday it would buy 50 Airbus 350-900 widebody aircraft in a $16 billion deal, as it reorganises its fleet after cutting orders of the A380 superjumbo.\n "I am happy to announce we are purchasing 50 Airbus 350 for $16 billion," the carrier\'s chairman and CEO Sheikh Ahmed bin Saeed Al-Maktoum told reporters at the Dubai Airshow, adding that delivery would begin from May 2023.\n Its move earlier this year to cut 39 aircraft from its total A380 orders prompted Airbus to pull the plug on the costly plane, which airlines have struggled to fill to its capacity of 500-850 people.\n The airline said that Monday\'s announcement "replaces the heads of agreement signed in February where Emirates announced its intent to purchase 30 A350s and 40 A330Neos."\n The airline said the A350 will enable it to serve a range of new markets, including on long-haul routes up to 15 hours of flying time from its base in Dubai.\n Emirates Airline, the largest carrier in the Middle East, re

In [217]:
text('Emirates')

3

In [216]:
dir(text)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'zfill']

In [204]:
scores_photos[959]

{'Spanish': 0.531914894,
 'Unai Emery': 0.5106382979999999,
 'Eintracht Frankfurt': 0.872340426,
 'Emirates': 0.872340426,
 'London': 0.872340426,
 'AFP / DANIEL LEAL-OLIVAS': 0.765957447}

In [205]:
scores_photos[330]

{"United Arab Emirates'": 0.425531915,
 'Aerobatic Team': 1.0,
 'Al-Fursan': 0.5106382979999999,
 'the Dubai Airshow': 0.872340426,
 'KARIM SAHIB': 0.021276596000000002}

In [186]:
score_sim3(scores_depeches[2], scores_photos[957])

0

In [167]:
photos.caption[rand_ind_list_photos[957]]

'CORRECTION / Pope Francis (L) sits with Japan\'s Prime Minister Shinzo Abe (R) during a meeting with the diplomatic community at the prime minister\'s official residence in Tokyo on November 25, 2019. Pope Francis called on November 25 for renewed efforts to help victims of Japan\'s 2011 "triple disaster" of earthquake, tsunami and the Fukushima meltdown, noting "concern" in the country over the continued use of nuclear power.  / AFP / Vincenzo PINTO'

In [175]:
score_sim3(scores_depeches[2], scores_photos[330])

0.964475257504108

In [190]:
for j in range(len(scores_photos)):
    if(set(photos.event[rand_ind_list_photos[j]][1:-1].replace("'","").split(', ')).intersection(set(depeches.event[rand_ind_list[2]][1:-1].replace("'","").split(', '))) != set()):
        print(j)

330
332
338
339
345
347
457
495
524
526
528
567
571
576
726


In [174]:
scores_photos[330]

{"United Arab Emirates'": 0.425531915,
 'Aerobatic Team': 1.0,
 'Al-Fursan': 0.5106382979999999,
 'the Dubai Airshow': 0.872340426,
 'KARIM SAHIB': 0.021276596000000002}